In [1]:
import pandas
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset


class MinstDataset(Dataset):
    
    def __init__(self, csv_file):
        self.data_df = pandas.read_csv(csv_file, header=None)
        
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, index):
        label = self.data_df.iloc[index, 0]
        target = torch.zeros(10)
        target[label] = 1.0
        
        image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values) / 255.0
        
        return label, image_values, target
    
    def plot_image(self, index):
        arr = self.data_df.iloc[index,1:].values.reshape(28,28)
        plt.title("label = " + str(self.data_df.iloc[index,0]))
        plt.imshow(arr, interpolation='none', cmap='Blues')

class Classifier(nn.Module):

    def __init__(self):
        super().__init__()
        
        # counter and accumulator for progress of training
        self.counter = 0
        self.progress = []
        
        # Init Neural Netowork model
        self.model = nn.Sequential(
                nn.Linear(784, 200),
                nn.LeakyReLU(),
                nn.LayerNorm(200),
                nn.Linear(200, 10),
                nn.Sigmoid()
        )
        
        # Loss function
        self.loss_function = nn.MSELoss()
        
        # Optimiser for backpropagation
        self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)
        
    def forward(self, inputs):
        return self.model(inputs)
    
    def train(self, inputs, targets):
        outputs = self.forward(inputs)
        loss = self.loss_function(outputs, targets)
        
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        
        self.counter +=1
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            
        if (self.counter % 10000 == 0):
            print("Counter = " + str(self.counter))

    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0, 1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))

In [ ]:
minst_dataset = MinstDataset('mnist_train_100.csv')
minst_dataset.plot_image(0)

In [ ]:
# Create NN and train it with epochs
%%time

c = Classifier()

epochs = 10

for i in range(epochs):
    print("epoch " + str(i))
    for label, image_data_tensor, target_tensor in minst_dataset:
        c.train(image_data_tensor, target_tensor)

In [ ]:
# Check loss function
c.plot_progress()

In [ ]:
mnist_test = MinstDataset('mnist_test_10.csv')
mnist_test.plot_image(5)

In [ ]:
# test NN on training data
score = 0
items = 0
for label, image_data_tensor, target_tensor in mnist_test:
    answer = c.forward(image_data_tensor).detach().numpy()
    if (answer.argmax() == label):
        score += 1
    items +=1
    
print(score, items, score/items)